In [1]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [3]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [4]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [5]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler = StandardScaler()

In [6]:
# Create a DataFrame with the scaled data
numeric_columns = df_market_data.select_dtypes(include=['float64', 'int64']).columns
scaled_df = pd.DataFrame(scaler.fit_transform(df_market_data[numeric_columns]), columns=numeric_columns)
# Copy the crypto names from the original data
scaled_df['crypto_name'] = scaled_df.index
# Set the coinid column as index
scaled_df.reset_index(inplace=True)
# Display sample data
print(scaled_df.head())

   index  price_change_percentage_24h  price_change_percentage_7d  \
0      0                     0.508529                    0.493193   
1      1                     0.185446                    0.934445   
2      2                     0.021774                   -0.706337   
3      3                    -0.040764                   -0.810928   
4      4                     1.193036                    2.000959   

   price_change_percentage_14d  price_change_percentage_30d  \
0                     0.772200                     0.235460   
1                     0.558692                    -0.054341   
2                    -0.021680                    -0.061030   
3                     0.249458                    -0.050388   
4                     1.760610                     0.545842   

   price_change_percentage_60d  price_change_percentage_200d  \
0                    -0.067495                     -0.355953   
1                    -0.273483                     -0.115759   
2             

---

### Find the Best Value for k Using the Original Data.

In [7]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1,12))
print(k_values)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [8]:
# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
for k in range(1,12):
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
    kmeans_model = KMeans(n_clusters=k, random_state=42)
# 2. Fit the model to the data using `df_market_data_scaled`
    kmeans_model.fit(scaled_df)
# 3. Append the model.inertia_ to the inertia list
    inertia_values.append(kmeans_model.inertia_)


c:\Users\joemd\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\joemd\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\joemd\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\joemd\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak

In [9]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = dict(zip(range(1,12), inertia_values))
# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(list(elbow_data.items()), columns=['Number of Clusters (k)', 'Inertia'])
print(elbow_df)

    Number of Clusters (k)       Inertia
0                        1  11767.000000
1                        2   3139.039244
2                        3   1531.663011
3                        4    968.577269
4                        5    715.555989
5                        6    551.991491
6                        7    471.158271
7                        8    402.791982
8                        9    326.782224
9                       10    261.472967
10                      11    208.602661


In [13]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot = elbow_df.hvplot.line(x='Number of Clusters (k)', y='Inertia', title='Elbow Curve for KMeans Clustering')
elbow_plot

:Curve   [Number of Clusters (k)]   (Inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** The best value for k is 2. We know this because at 2 clusters, we can visuazlly see that this is where the elbow curve begins to level out. This is because the intertia is beginning to slow down. 

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [19]:
# Initialize the K-Means model using the best value for k
best_k = 2
kmeans_model = KMeans(n_clusters=best_k, random_state=30)

In [20]:
# Fit the K-Means model using the scaled data
kmeans_model.fit(scaled_df)

c:\Users\joemd\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\joemd\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=2, random_state=30)

In [21]:
# Predict the clusters to group the cryptocurrencies using the scaled data
predict_clusters = kmeans_model.predict(scaled_df)
# Print the resulting array of cluster values.
print(predict_clusters)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]


In [22]:
# Create a copy of the DataFrame
copy_df = scaled_df.copy()

In [23]:
# Add a new column to the DataFrame with the predicted clusters
copy_df['predicted_cluster'] = predict_clusters
# Display sample data
print(copy_df.head())

   index  price_change_percentage_24h  price_change_percentage_7d  \
0      0                     0.508529                    0.493193   
1      1                     0.185446                    0.934445   
2      2                     0.021774                   -0.706337   
3      3                    -0.040764                   -0.810928   
4      4                     1.193036                    2.000959   

   price_change_percentage_14d  price_change_percentage_30d  \
0                     0.772200                     0.235460   
1                     0.558692                    -0.054341   
2                    -0.021680                    -0.061030   
3                     0.249458                    -0.050388   
4                     1.760610                     0.545842   

   price_change_percentage_60d  price_change_percentage_200d  \
0                    -0.067495                     -0.355953   
1                    -0.273483                     -0.115759   
2             

In [24]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
scatter = copy_df.hvplot.scatter(
    x='price_change_percentage_24h',
    y='price_change_percentage_7d',
    c='predicted_cluster',  
    cmap='viridis', 
    size=12,  
    hover_cols=['coin_id'],  
    title='Crypto Clustering Scatter Plot'
)

scatter

:Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,predicted_cluster)

---

### Optimize Clusters with Principal Component Analysis.

In [25]:
# Create a PCA model instance and set `n_components=3`.
pca_model = PCA(n_components=3)

In [27]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
principal_components = pca_model.fit_transform(scaled_df)
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2', 'PC3'])
# View the first five rows of the DataFrame. 
print(pca_df.head())

         PC1       PC2       PC3
0  28.301603  0.527108 -0.125684
1  26.886951  0.340747  0.262465
2  25.435524 -0.192330  0.416851
3  24.029431 -0.261087  0.423343
4  22.711517  1.061241 -1.262679


In [28]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
explained_variance = pca_model.explained_variance_ratio_
for i, variance in enumerate(explained_variance, start=1):
    print(f"Explained Vairance for PC{i}: {variance:.4f}")

Explained Vairance for PC1: 0.9773
Explained Vairance for PC2: 0.0086
Explained Vairance for PC3: 0.0079


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained variance of the 3 principal components is 99.38%. 

In [29]:
# Create a new DataFrame with the PCA data.
pca_result_df = pd.DataFrame(data=principal_components, columns=['PC1','PC2','PC3'])
# Creating a DataFrame with the PCA data
# Copy the crypto names from the original data
pca_result_df.index = df_market_data.index
# Set the coinid column as index
# Display sample data
print(pca_result_df.head())

                    PC1       PC2       PC3
coin_id                                    
bitcoin       28.301603  0.527108 -0.125684
ethereum      26.886951  0.340747  0.262465
tether        25.435524 -0.192330  0.416851
ripple        24.029431 -0.261087  0.423343
bitcoin-cash  22.711517  1.061241 -1.262679


---

### Find the Best Value for k Using the PCA Data

In [30]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1,12))

In [31]:
# Create an empty list to store the inertia values
inertia_values = []
# Create a for loop to compute the inertia with each possible value of k
for k in k_values:
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
    kmeans_model = KMeans(n_clusters=k, random_state=30)
# 2. Fit the model to the data using `df_market_data_pca`
    kmeans_model.fit(pca_result_df)
# 3. Append the model.inertia_ to the inertia list
    inertia_values.append(kmeans_model.inertia_)
print(inertia_values)


c:\Users\joemd\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\joemd\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\joemd\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\joemd\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak

[11695.03760341048, 3067.224245421208, 1461.254029608378, 903.8272048361757, 637.8081521221211, 485.3708472508036, 399.8167697440686, 337.83780825261414, 264.1750924489548, 203.6851046858954, 165.49037230914107]


In [32]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data2 = {'Number of Clusters (k)': k_values, 'Inertia': inertia_values}
# Create a DataFrame with the data to plot the Elbow curve
elbow_df2 = pd.DataFrame(elbow_data2)
print(elbow_df2)

    Number of Clusters (k)       Inertia
0                        1  11695.037603
1                        2   3067.224245
2                        3   1461.254030
3                        4    903.827205
4                        5    637.808152
5                        6    485.370847
6                        7    399.816770
7                        8    337.837808
8                        9    264.175092
9                       10    203.685105
10                      11    165.490372


In [34]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot2 = elbow_df2.hvplot.line(x='Number of Clusters (k)', y='Inertia', title='Elbow Curve for KMeans Clustering')
elbow_plot

:Curve   [Number of Clusters (k)]   (Inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 2 is the best value for k when using the PCA data. 


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, the results from using the PCA data do not differ when compared with the best k value found when using the original data. 

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [35]:
# Initialize the K-Means model using the best value for k
best_k = 2

In [37]:
# Fit the K-Means model using the PCA data
kmeans_model.fit(pca_result_df)

c:\Users\joemd\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\joemd\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=11, random_state=30)

In [38]:
# Predict the clusters to group the cryptocurrencies using the PCA data
predict_clusters_pca = kmeans_model.predict(pca_result_df)
# Print the resulting array of cluster values.
print(predict_clusters_pca)

[ 6  6  6  6  3  3  3  3  3  1  1  1  1  1  1  9  9  9  9  2  2  2  2  8
  8  8  8  4  4  4  4  4 10  0  0  0  5  7  7  7  7]


In [39]:
# Create a copy of the DataFrame with the PCA data
pca_result_df_copy = pca_result_df.copy()
# Add a new column to the DataFrame with the predicted clusters
pca_result_df_copy['Cluster'] = predict_clusters_pca
# Display sample data
print(pca_result_df_copy.head())

                    PC1       PC2       PC3  Cluster
coin_id                                             
bitcoin       28.301603  0.527108 -0.125684        6
ethereum      26.886951  0.340747  0.262465        6
tether        25.435524 -0.192330  0.416851        6
ripple        24.029431 -0.261087  0.423343        6
bitcoin-cash  22.711517  1.061241 -1.262679        3


In [40]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
scatter2 = pca_result_df_copy.hvplot.scatter(
    x='PC1',
    y='PC2',
    c='Cluster',  
    colormap='viridis',  
    size=10,  
    hover_cols=['coin_id'],  
    title='PCA Scatter Plot',
    xlabel='Principal Component 1 (PC1)',
    ylabel='Principal Component 2 (PC2)',
    legend='bottom_right'
)

scatter2

:Scatter   [PC1]   (PC2,Cluster,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [41]:
# Composite plot to contrast the Elbow curves
elbow_plot + elbow_plot2

:Layout
   .Curve.I  :Curve   [Number of Clusters (k)]   (Inertia)
   .Curve.II :Curve   [Number of Clusters (k)]   (Inertia)

In [42]:
# Composite plot to contrast the clusters
scatter + scatter2

:Layout
   .Scatter.I  :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,predicted_cluster)
   .Scatter.II :Scatter   [PC1]   (PC2,Cluster,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** The impact of using fewer features to cluster the data using K-Means is that the intertia descrease and the data points appeared to be less dispesred (or more tightly clustered). When we have fewer features to cluster the data, this seems to reduce ambiguity and in turn lends itself to better interpretation of results. 